In [1]:
import pprint

import numpy as np
import pandas as pd

from sklearn import preprocessing, metrics

import torch
from torch import Tensor
from torch.utils.data import DataLoader
from torch.optim import AdamW

import torchtext
import torchtext.transforms as T
import torchtext.functional as F
from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER

import pickle

In [2]:
train_df = pd.read_json('/kaggle/input/ds-dataset/sensitive_train.json')

In [3]:
label_encoder = preprocessing.LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])

In [4]:
test_df = pd.read_json('/kaggle/input/ds-dataset/sensitive_test.json')

In [5]:
test_df['label'] = label_encoder.transform(test_df['label'])

In [6]:
class PadTransform(torch.nn.Module):
    """Pad tensor to a fixed length with given padding value.

    :param max_length: Maximum length to pad to
    :type max_length: int
    :param pad_value: Value to pad the tensor with
    :type pad_value: bool
    """

    def __init__(self, max_length: int, pad_value: int) -> None:
        super().__init__()
        self.max_length = max_length
        self.pad_value = float(pad_value)
        
    def forward(self, x: Tensor) -> Tensor:
        """
        :param x: The tensor to pad
        :type x: Tensor
        :return: Tensor padded up to max_length with pad_value
        :rtype: Tensor
        """
        max_encoded_length = x.size(-1)
        if max_encoded_length < self.max_length:
            pad_amount = self.max_length - max_encoded_length
            x = torch.nn.functional.pad(x, (0, pad_amount), value=self.pad_value)
        return x

In [7]:
padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256

In [8]:
text_transform = torchtext.models.XLMR_LARGE_ENCODER.transform()

100%|██████████| 5.07M/5.07M [00:00<00:00, 5.23MB/s]
Downloading: "https://download.pytorch.org/models/text/xlmr.vocab.pt" to /root/.cache/torch/hub/checkpoints/xlmr.vocab.pt


  0%|          | 0.00/4.85M [00:00<?, ?B/s]

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return (
            torch.tensor(text_transform(self.df.iloc[idx, 0])),
            torch.tensor(self.df.iloc[idx, 1])
        )

In [10]:
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

In [11]:
SMALL_BATCH_SIZE = 16
K = 2
LARGE_BATCH_SIZE = K * SMALL_BATCH_SIZE

In [12]:
def batch_collate_fn(batch):
    inp_list = list()
    tar_list = list()
    
    for sample in batch:
        inp_list.append(sample[0].tolist())
        tar_list.append(sample[1])
        
    padded_tensor = F.to_tensor(inp_list, padding_value=padding_idx)
    target_tensor = torch.stack(tar_list).type(torch.LongTensor)
    
    return padded_tensor, target_tensor

In [13]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=SMALL_BATCH_SIZE,
    shuffle=True,
    collate_fn=batch_collate_fn
)

In [14]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=SMALL_BATCH_SIZE,
    shuffle=False,
    collate_fn=batch_collate_fn
)

In [15]:
num_classes = 5
input_dim = 768

In [16]:
classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)

Downloading: "https://download.pytorch.org/models/text/xlmr.base.encoder.pt" to /root/.cache/torch/hub/checkpoints/xlmr.base.encoder.pt


  0%|          | 0.00/1.03G [00:00<?, ?B/s]

In [17]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [18]:
model.to(DEVICE)
pass  # prevent printing

In [19]:
learning_rate = 1.2e-5 
optim = AdamW(model.parameters(), lr=learning_rate)
criteria = torch.nn.CrossEntropyLoss()

In [20]:
def train_step(input, target, small_batch_no):
    output = model(input)
    loss = criteria(output, target)
    loss.backward()
    
    if (small_batch_no + 1) % K == 0 or (small_batch_no + 1) == len(train_dataloader):
        optim.step()
        optim.zero_grad()
    
    return loss.item() / input.size(dim=0)

def evaluate():
    model.eval()
    
    total_loss = 0
    counter = 0
    
    with torch.no_grad():
        for i, batch in enumerate(test_dataloader):
            input = batch[0].clone().detach().to(DEVICE)
            output = model(input)
            target = batch[1].clone().detach().to(DEVICE)
            
            if i == 0:
                class_output = torch.argmax(output, dim=1)
                class_target = target
            else:
                class_output = torch.cat([class_output, torch.argmax(output, dim=1)])
                class_target = torch.cat([class_target, target])
            
            loss = criteria(output, target).item()
            total_loss += loss
            
            counter += input.size(dim=0)
            
            
        confusion_matrix = metrics.confusion_matrix(
            class_target.cpu().numpy().flatten(),
            class_output.cpu().numpy().flatten(),
            labels=[0, 1, 2, 3, 4]
        ) 
        classification_report = metrics.classification_report(
            class_target.cpu().numpy().flatten(),
            class_output.cpu().numpy().flatten(),
            labels=[0, 1, 2, 3, 4],
            output_dict=True
        )

    return (
        total_loss,
        counter,
        confusion_matrix,
        classification_report
    )

In [21]:
num_epochs = 500

In [22]:
def save_log(epoch, **kwargs):
    with open(f'log_{epoch}.pkl', 'wb') as f:
        pickle.dump(kwargs, f)

def save_model(message):
    torch.save(model.state_dict(), f'model_{message}.pth')
    print('Model saved successfully')

In [23]:
max_accuracy = float('-inf')
max_macro_f1 = float('-inf')
max_weighted_f1 = float('-inf')

In [24]:
for epoch in range(num_epochs):
    model.train()
    
    avg_training_losses = list()
    
    for small_batch_no, small_batch in enumerate(train_dataloader):
        input = small_batch[0].clone().detach().to(DEVICE)
        target = small_batch[1].clone().detach().to(DEVICE)
        avg_training_losses.append(
            train_step(input, target, small_batch_no)
        )
        
        torch.cuda.empty_cache()

    total_loss, counter, confusion_matrix, classification_report = evaluate()
    print(f'EPOCH {epoch}')
    print(f'Mean of avg_training_losses={np.mean(avg_training_losses)}')
    print(f'total_loss={total_loss}')
    print(f'counter={counter}')
    print(f'loss=total_loss/counter={total_loss/counter}')
    print(f'confusion_matrix=\n{confusion_matrix}')
    
    print('classification_report=')
    pprint.pprint(classification_report)
    
    save_log(
        epoch,
        avg_training_losses=avg_training_losses,
        total_loss=total_loss,
        counter=counter,
        loss=total_loss/counter,
        confusion_matrix=confusion_matrix,
        classification_report=classification_report
    )
    
    if classification_report['accuracy'] > max_accuracy:
        print(f'New max_accuracy')
        max_accuracy = classification_report['accuracy']
        max_accuracy_index = epoch
        save_model('max_accuracy')
        
    elif classification_report['macro avg']['f1-score'] > max_macro_f1:
        print(f'New max_macro_f1')
        max_macro_f1 = classification_report['macro avg']['f1-score']
        max_macro_f1_index = epoch
        save_model('max_macro_f1')
    
    elif classification_report['weighted avg']['f1-score'] > max_weighted_f1:
        print(f'New max_weighted_f1')
        max_weighted_f1 = classification_report['weighted avg']['f1-score']
        max_weighted_f1_index = epoch
        save_model('max_weighted_f1')
    
    elif epoch == num_epochs - 1:
        save_model(f'{epoch}_last')
    
    elif epoch % 40 == 0:
        save_model(f'{epoch}_checkpoint')
        
    else:
        continue
    

save_log(
    'post_train_info',
    max_accuracy=max_accuracy,
    max_accuracy_index=max_accuracy_index,
    max_macro_f1=max_macro_f1,
    max_macro_f1_index=max_macro_f1_index,
    max_weighted_f1=max_weighted_f1,
    max_weighted_f1_index=max_weighted_f1_index
)

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/skle

EPOCH 0
Mean of avg_training_losses=0.09712835365078516
total_loss=93.32161903381348
counter=1019
loss=total_loss/counter=0.09158156921865895
confusion_matrix=
[[  0 188   0   0  17]
 [  0 227   0   0  43]
 [  0  96   0   0  78]
 [  0 139   0   0  20]
 [  0   6   0   0 205]]
classification_report=
{'0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 205},
 '1': {'f1-score': 0.49028077753779703,
       'precision': 0.34603658536585363,
       'recall': 0.8407407407407408,
       'support': 270},
 '2': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 174},
 '3': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 159},
 '4': {'f1-score': 0.7142857142857142,
       'precision': 0.5647382920110193,
       'recall': 0.9715639810426541,
       'support': 211},
 'accuracy': 0.4239450441609421,
 'macro avg': {'f1-score': 0.24091329836470227,
               'precision': 0.18215497547537457,
               'recall': 0.36246094435667897,
               'suppo

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


EPOCH 1
Mean of avg_training_losses=0.07633925842431684
total_loss=68.42765238881111
counter=1019
loss=total_loss/counter=0.06715176878195399
confusion_matrix=
[[ 88 109   0   2   6]
 [  0 197  34  14  25]
 [  0  18  85   0  71]
 [  0  37   0 115   7]
 [  0   4   1   0 206]]
classification_report=
{'0': {'f1-score': 0.6006825938566553,
       'precision': 1.0,
       'recall': 0.4292682926829268,
       'support': 205},
 '1': {'f1-score': 0.620472440944882,
       'precision': 0.5397260273972603,
       'recall': 0.7296296296296296,
       'support': 270},
 '2': {'f1-score': 0.5782312925170068,
       'precision': 0.7083333333333334,
       'recall': 0.4885057471264368,
       'support': 174},
 '3': {'f1-score': 0.7931034482758621,
       'precision': 0.8778625954198473,
       'recall': 0.7232704402515723,
       'support': 159},
 '4': {'f1-score': 0.7832699619771863,
       'precision': 0.653968253968254,
       'recall': 0.976303317535545,
       'support': 211},
 'accuracy': 0.6781

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


...

EPOCH 497
Mean of avg_training_losses=2.401416598413814e-06
total_loss=182.12339882454853
counter=1019
loss=total_loss/counter=0.17872757490142152
confusion_matrix=
[[178  23   1   2   1]
 [  3 134  97  21  15]
 [  0   8 122   1  43]
 [  0  37   0 122   0]
 [  0   2   5   1 203]]
classification_report=
{'0': {'f1-score': 0.9222797927461139,
       'precision': 0.9834254143646409,
       'recall': 0.8682926829268293,
       'support': 205},
 '1': {'f1-score': 0.5654008438818565,
       'precision': 0.6568627450980392,
       'recall': 0.4962962962962963,
       'support': 270},
 '2': {'f1-score': 0.6115288220551378,
       'precision': 0.5422222222222223,
       'recall': 0.7011494252873564,
       'support': 174},
 '3': {'f1-score': 0.7973856209150327,
       'precision': 0.8299319727891157,
       'recall': 0.7672955974842768,
       'support': 159},
 '4': {'f1-score': 0.8583509513742072,
       'precision': 0.7748091603053435,
       'recall': 0.9620853080568721,
       'support': 21

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


EPOCH 498
Mean of avg_training_losses=2.132381372415498e-06
total_loss=181.14616973653187
counter=1019
loss=total_loss/counter=0.17776856696421184
confusion_matrix=
[[178  23   1   2   1]
 [  3 135  93  24  15]
 [  0   8 122   1  43]
 [  0  41   0 118   0]
 [  0   2   5   1 203]]
classification_report=
{'0': {'f1-score': 0.9222797927461139,
       'precision': 0.9834254143646409,
       'recall': 0.8682926829268293,
       'support': 205},
 '1': {'f1-score': 0.5636743215031315,
       'precision': 0.645933014354067,
       'recall': 0.5,
       'support': 270},
 '2': {'f1-score': 0.6177215189873418,
       'precision': 0.5520361990950227,
       'recall': 0.7011494252873564,
       'support': 174},
 '3': {'f1-score': 0.7737704918032786,
       'precision': 0.8082191780821918,
       'recall': 0.7421383647798742,
       'support': 159},
 '4': {'f1-score': 0.8583509513742072,
       'precision': 0.7748091603053435,
       'recall': 0.9620853080568721,
       'support': 211},
 'accuracy':

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


EPOCH 499
Mean of avg_training_losses=1.6781724535934e-05
total_loss=181.6455852834331
counter=1019
loss=total_loss/counter=0.17825867054311392
confusion_matrix=
[[178  23   1   2   1]
 [  3 136  92  24  15]
 [  0   9 122   1  42]
 [  0  41   0 118   0]
 [  0   2   5   1 203]]
classification_report=
{'0': {'f1-score': 0.9222797927461139,
       'precision': 0.9834254143646409,
       'recall': 0.8682926829268293,
       'support': 205},
 '1': {'f1-score': 0.5654885654885655,
       'precision': 0.6445497630331753,
       'recall': 0.5037037037037037,
       'support': 270},
 '2': {'f1-score': 0.6192893401015228,
       'precision': 0.5545454545454546,
       'recall': 0.7011494252873564,
       'support': 174},
 '3': {'f1-score': 0.7737704918032786,
       'precision': 0.8082191780821918,
       'recall': 0.7421383647798742,
       'support': 159},
 '4': {'f1-score': 0.8601694915254239,
       'precision': 0.7777777777777778,
       'recall': 0.9620853080568721,
       'support': 211},